In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import equationsH as eq

from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from scipy import optimize

In [ ]:
# shooting
# with polytropo
def shootingP(p_i, sig_i, Up_, Low_, rmax_, rmin_, L3_, c4_, k_=100, L_=2):
    """
    este es con las ecuaciones con polítropo
    N, g, a, b, p -> N, g, phi, phi', p
    """
    p0_, sig0_ = p_i, sig_i
    U0 = [1, 1, sig0_, 0, p0_]
    
    # IMPORTANT: it is not possible to find two event at same time
    def negSig(r, U, c): return U[2]
    def posSig(t, U, c): return U[-2]  # we used phi' as condition to find when phi grows
    # neg
    negSig.terminal = True; negSig.direction = -1
    # pos
    posSig.terminal = True; posSig.direction = 1
  
    while True:
        w_ = (Up_+Low_)/2
        c_ = [w_, c4_, L3_, L_, k_, p0_]
        
        sol_ = solve_ivp(eq.ODE_eqP, [rmin_, rmax_], U0, args=([c_]), events=(posSig, negSig))
        
        if sol_.t_events[0].size != 0:  # 0 -> no event, 1 -> happened the event
            # print('positivo')
            Low_ = w_
            rTemp_ = sol_.t_events[0]
        elif sol_.t_events[1].size != 0: 
            # print('negativo')
            Up_ = w_
            rTemp_ = sol_.t_events[1]
        else:  # if it not have event in all r-interval, the w is fine
            print('Encontrado', w_)
            return w_, [rmax_]
        
        # checking the lim freq. 
        if abs((Up_-Low_)/2)<=1e-15:
            print('Maxima presición alcanzada', w_, 'radio', rTemp_)
            return w_, rTemp_
        
        
# shooting constante        
def shootingC(p_i, sig_i, Up_, Low_, rmax_, rho_, rmin_, L3_, c4_):
    """
    para rho=cte
    # N, g, a, b, p -> N, g, phi, phi', p
    """
    p0_, sig0_ = p_i, sig_i
    U0 = [1, 1, sig0_, 0, p0_]
    
    # IMPORTANT: it is not possible to find two event at same time
    def negSig(r, U, c): return U[2]
    def posSig(t, U, c): return U[-2]  # we used phi' as condition to find when phi grows
    # neg
    negSig.terminal = True; negSig.direction = -1
    # pos
    posSig.terminal = True; posSig.direction = 1
    
    while True:
        w_ = (Up_+Low_)/2   
        c_ = [w_, c4_, L3_, rho_, p0_]
        
        sol_ = solve_ivp(eq.ODE_eq, [rmin_, rmax_], U0, args=([c_]), events=(posSig, negSig))
        
        if sol_.t_events[0].size != 0:  # 0 -> no event, 1 -> happened the event
            # print('positivo')
            Low_ = w_
            rTemp_ = sol_.t_events[0]
        elif sol_.t_events[1].size != 0: 
            # print('negativo')
            Up_ = w_
            rTemp_ = sol_.t_events[1]
        else:  # if it not have event in all r-interval, the w is fine
            print('Encontrado', w_)
            return w_, [rmax_]
        
        # checking the lim freq. 
        if abs((Up_-Low_)/2)<=4e-16:
            print('Maxima presición alcanzada', w_, 'radio', rTemp_)
            return w_, rTemp_

In [ ]:
# mass definition
M = lambda r, g: 4*np.pi*(r*(1-1/g**2))

## Without polytropo

In [ ]:
# Example with constant density 
#start_time = time.time()
# N, g, a, b, p = U

# parameters

rmax = 20 #20 #35
L3 = 1. # 1.,  1.2,  1.5
c4 = 1/2 # 0, 1/2, -1/2
rho = 0.01
sig0 = 0.6 #0.25
rmin= 1e-01

Up =  3.9805761#1.6
Low = 2.7106431991811903#1.2
#np.linspace(0.001, 0.12, 15)#[0.00381966]
p_val =  [.5]
#[0.025118864315095808, 0.6, 0.01, 2.4904823548311823, 1.0, 0.5, 0.1, 12],
# [0.05011872336272725, 0.6, 0.01, 2.508677603827091, 1.0, 0.5, 0.1, 12],
#np.logspace(-1.9, -1, 4) #np.logspace(-4, -0.92, 30)

dat = []

#(p_i, sig_i, Up_, Low_, rmax_, rho_, rmin_, L3_, c4_)
for p0 in p_val:
    #print(p0)
    w, rfind = shootingC(p0, sig0, Up, Low, rmax_=rmax, rho_=rho, rmin_=rmin, L3_=L3, c4_=c4)
    #print("--- %s seconds ---" % (time.time() - start_time))
    dat.append([p0, sig0, rho, w, L3, c4, rmin, rfind[0]])

In [ ]:
# generando datos    
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(9,4))

def zeroP(r, U, c): return U[4]
zeroP.direction = -1

result = []
for i in dat: #025P
    p0, sig0, rho, w, L3, c4, rmin, rmax = i
    #print(p0)
    c = [w, c4, L3, rho, p0]  # w, c4, L3, rho
    rspan = np.linspace(rmin, rmax, 300)
    U0 = [1, 1, sig0, 0, p0]
    
    sol = solve_ivp(eq.ODE_eq, [rspan[0], rspan[-1]], U0, t_eval=rspan, args=([c]),
               events=(zeroP))  # rtol = 1e-5

    print(sol.t_events, rmax)
    
    #np.savetxt(f'ML1_{c4}.dat', np.transpose([sol.t, sol.y[1]]) , delimiter=' ') 
    
    m99 = 0.99*M(sol.t[-1], sol.y[1][-1])
    Mass_f = interp1d(sol.t,  M(sol.t, sol.y[1]))

    func = lambda r: Mass_f(r)-m99

    r99 = optimize.root_scalar(func, bracket=[sol.t[0], sol.t[-1]], method='brentq')

    #result.append([r99.root, m99, (m99/(8*np.pi*r99.root)), p0, sig0, rho])
    result.append([p0, (m99/(8*np.pi*r99.root))])

    # Plotting the results
    ax[0].plot(sol.t, sol.y[-1]/sol.y[-1][0], c='blue', label='pressure')
    ax[0].plot(sol.t, sol.y[2]/sol.y[2][0], c='black', label='sigma')

    ax[0].vlines(x=sol.t_events, ymin=0, ymax=1, ls=':', alpha=0.4, color='black')
    #ax[0].set_yscale('log')

    ax[1].plot(sol.t, M(sol.t, sol.y[1]), label='mass')
    ax[1].plot(r99.root, m99,'o')
    ax[1].vlines(x=sol.t_events, ymin=0, ymax=200, ls=':', alpha=0.4, color='black')

#ax[0].legend(loc='upper center')
#ax[1].legend(loc='upper center')

#ax[0].set_ylim(0,1.1)
#ax[0].set_xlim(0, rmax)
    

## With polytropo

In [ ]:
# Example with constant density 
#start_time = time.time()
# N, g, a, b, p = U

# parameters

rmax = 40
L3 = 1 # 1.,  1.2,  1.5
c4 = -1/2 # 0, 1/2, -1/2
#rho = 0.01
sig0 = 0.25 # 0.25, 0.05
rmin = 1e-02
k = 100
L = 2

Up = 1.38
Low = 1.2709306349152583
#np.linspace(0.001, 0.12, 15)
#p_val = np.logspace(-4, -0.92, 30) # [0.00381966]

p_val = np.logspace(np.log10(.005), -0.92, 30) #np.logspace(-6, -2, 29)

dat = []

for p0 in p_val:
    print(p0)
    w, rfind = shootingP(p0, sig0, Up, Low, rmax_=rmax, rmin_=rmin, L3_=L3, c4_=c4, k_=k, L_=L)
    #print("--- %s seconds ---" % (time.time() - start_time))
    dat.append([p0, sig0, k, L, w, L3, c4, rmin, rfind[0]])

In [ ]:
# generando datos    
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(9,4))

def zeroP(r, U, c): return U[4]
zeroP.direction = -1

result = []
for i in dat: #025P
    p0, sig0, k, L, w, L3, c4, rmin, rmax = i
    #print(p0)
    c = [w, c4, L3, L, k, p0]  # w, c4, L3, rho
    rspan = np.linspace(rmin, rmax, 300)
    U0 = [1, 1, sig0, 0, p0]
      
    sol = solve_ivp(eq.ODE_eqP, [rspan[0], rspan[-1]], U0, t_eval=rspan, args=([c]),
               events=(zeroP))  # rtol = 1e-5

    print(sol.t_events, rmax, p0/1e04)
    
    #np.savetxt(f'ML1_{c4}.dat', np.transpose([sol.t, sol.y[1]]) , delimiter=' ') 
    
    m99 = 0.99*M(sol.t[-1], sol.y[1][-1])
    Mass_f = interp1d(sol.t,  M(sol.t, sol.y[1]))

    func = lambda r: Mass_f(r)-m99

    r99 = optimize.root_scalar(func, bracket=[sol.t[0], sol.t[-1]], method='brentq')

    #result.append([r99.root, m99, (m99/(8*np.pi*r99.root)), p0, sig0, rho])
    result.append([p0, (m99/(8*np.pi*r99.root))])
    #result.append([p0, m99, r99.root])
    # Plotting the results
    ax[0].plot(sol.t, sol.y[-1]/sol.y[-1][0], c='blue', label='pressure')
    ax[0].plot(sol.t, sol.y[2]/sol.y[2][0], c='black', label='sigma')

    ax[0].vlines(x=sol.t_events, ymin=0, ymax=1, ls=':', alpha=0.4, color='black')
    ax[0].set_yscale('log')

    ax[1].plot(sol.t, M(sol.t, sol.y[1]))#/(8*np.pi*sol.t), label='mass')
    #ax[1].plot(r99.root, m99,'o')
    #ax[1].vlines(x=sol.t_events, ymin=0, ymax=1, ls=':', alpha=0.4, color='black')

#ax[0].legend(loc='upper center')
#ax[1].legend(loc='upper center')

#ax[0].set_ylim(0,1.1)
#ax[0].set_xlim(0, rmax)

In [ ]:
# plot 
# Example with polytropo
start_time = time.time()

# N, g, a, b, p = U

# parameters
rmin = 1e-3
rmax = 35
#L3 = 1.5
#c4 = -1/2
#k = 100
#L = 2 # gamma
p0 = 0.00381966
sig0 = 0.25


Up = 1.26
Low = 1.215

w = shootingP(p0, sig0, Up, Low, rmax, rmin_=1e-01, L3_=1.5, c4_=1/2, k_=100, L_=2)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Computing with the previusly results

# N, g, a, b, p = U

# parameters
rmin = 1e-1
rmax = 37
L3 = 1.5
c4 = 0 # 1/2
k = 100
L = 2 # gamma
p0 = 0.00381966
sig0 = 0.25

w = 1.2406958928744718

# %% Define time spans, initial values, and constants
c = [w, c4, L3, L, k, p0]
rspan = np.linspace(rmin, rmax, 300)
# N, g, a, b, p -> N, g, phi, phi', p
U0 = [1, 1, sig0, 0, p0]

def zeroP(r, U, c): return U[4]
#def zeroSig(r, U, c): return U[4]
zeroP.direction = -1
#zeroSig.terminal = True

sol = solve_ivp(eq.ODE_eqP, [rspan[0], rspan[-1]], U0, t_eval=rspan, args=([c]),
               events=(zeroP))  # rtol = 1e-5

print(sol.t_events)

In [ ]:
# Plotting the results
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(9,4))

ax[0].plot(sol.t, sol.y[-1]/sol.y[-1][0], label='pressure')
ax[0].plot(sol.t, sol.y[2]/sol.y[2][0], label='sigma')

ax[0].vlines(x=sol.t_events, ymin=0, ymax=1, ls=':', alpha=0.4, color='black')
ax[0].hlines(y=p0/1e04, xmin=0, xmax=rmax, ls=':', alpha=0.4, color='red')
ax[0].set_yscale('log')

ax[1].plot(sol.t, M(sol.t, sol.y[1]), label='mass')
ax[1].vlines(x=sol.t_events, ymin=0, ymax=200, ls=':', alpha=0.4, color='black')

ax[0].legend(loc='upper center')
ax[1].legend(loc='upper center')